<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

## Preliminaries

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
#!usr/bin/python3
!pip install pandas, bs4, requests, datetime, time, json
import pandas as pd
# import numpy as np
from bs4 import BeautifulSoup
# from ast import literal_eval
import requests, datetime, time
import json

ERROR: Invalid requirement: 'pandas,'


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Bring in auctions html

headers = {
    'User-Agent': 'AV User Agent 20201011',
    'From': 'anvance@gmail.com'
}

site = requests.get('https://bringatrailer.com/auctions', headers=headers)
site.status_code

In [ ]:
# Soup it
soup = BeautifulSoup(site.text, 'html.parser')

In [ ]:
# parse and build tuples for a table

labels = ['uid',
          'listing_title', 
          'listing_excerpt', 
          'listing_link', 
          'latest_bid', 
          'year',
          'make',
          'model',
          'category',
          'keyword',
          'vin',
          'latitude',
          'longitude']

listing_tuples = []

In [ ]:
for item in soup.findAll('div', class_ = lambda x: (x == 'auctions-item-container'))[1:]:
    try:
        uid = item['data-update']    
    except AttributeError:
        uid = 'not found'
    try:
        listing_title = item.find('h3',class_='auctions-item-title').text    
    except AttributeError:
        listing_title = 'not found'
    try:
        listing_excerpt = item.find('div', class_='auctions-item-excerpt').text
    except AttributeError:
        listing_excerpt = 'not found'
    try:
        listing_link = item.find('h3').find('a', attrs = {'href': True})['href']
    except AttributeError:
        listing_link = 'not found'
    try:
        latest_bid = item.find_all('span', attrs= {'data-current': True})[0]['data-current']
    except AttributeError:
        latest_bid = 'not found'
    try:
        year = json.loads(item['data-searchable'])['year']
    except AttributeError:
        year = 'not found'
    try:
        make = json.loads(item['data-searchable'])['make']
    except AttributeError:
        make = 'not found'
    try:
        model = json.loads(item['data-searchable'])['model']
    except AttributeError:
        model = 'not found'
    try:
        category = json.loads(item['data-searchable'])['category']
    except AttributeError:
        category = 'not found'
    try:
        keyword = json.loads(item['data-searchable'])['keyword']
    except AttributeError:
        keyword = 'not found'
    try:
        vin = json.loads(item['data-searchable'])['vin']
    except AttributeError:
        vin = 'not found'
    try:
        latitude = item['data-lat']
    except AttributeError:
        latitude = 'not found'
    try:
        longitude = item['data-lon']
    except AttributeError:
        longitude = 'not found'
    listing_tuple = (uid,
                     listing_title, 
                     listing_excerpt, 
                     listing_link, 
                     latest_bid, 
                     year,
                     make,
                     model,
                     category,
                     keyword,
                     vin,
                     latitude,
                     longitude)
    listing_tuples.append(listing_tuple)

In [ ]:
staging_df = pd.DataFrame(listing_tuples, columns = labels)

In [ ]:
time_string = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d_%H-%M-%S')
time_string_2 = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')

In [ ]:
staging_df['scrape_ts'] = time_string_2

In [ ]:
staging_df.to_csv('../Data/bat_listings_output_%s.csv' % time_string, index=True)